In [14]:
'''수입원장에서 피스토스 수금수수료 명세를 추출합니다 '''
import pandas as pd
import GladUtil as gd

ym  = '202409'
# dir0  = f'E:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
# dir0 = f'd:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
dir0 = f'N:/. Personal_folder/내 문서/GLAD/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
         
file0 = f'{ym}_수입원장.xlsx'  #원본


In [15]:
# 가온 수금명세 추출
brh = '글래드가온'  #지점
ff = dir0+file0
#피스토스 모집건 제외 조건: 사번 일치 + 20181101 이전계약
fc_code = ['GLD803173002','GLD033143019','GLD803141001','GLD803143004','GLD033143015','GLD033143010','GLD803153002']
c_date='20181101'

df = pd.read_excel(ff, sheet_name='피스토스수금', header=3, usecols='a:c').iloc[:12]

df_all = pd.DataFrame()
for i, row in df.iterrows():
    co_name = row['보험사']
    col = row['수금 값열']
    col_name = row['수금값 제목']
    # print(co_name, col)
    if co_name != '계':
       df_list = pd.read_excel(ff, sheet_name=co_name, usecols=f'a:ah,{col}', dtype={'계약일자':'object'})
       df_list = df_list[(df_list['지점'] == brh) & (df_list[col_name] != 0)].copy()
       df_list = df_list[~((df_list['사원번호'].isin(fc_code)) & (df_list['계약일자']<c_date))].copy().rename(columns={col_name:'수금수수료'})
       df_all = pd.concat([df_all, df_list])
       print(co_name, df_list['수수료계'].sum(), df_list['수금수수료'].sum())

df_sum = df_all.groupby(['보험사'])['수금수수료'].sum().reset_index()
df_sum['지급액'] = round(df_sum['수금수수료'] * 0.95,0)
df_all.drop(columns='수수료계',inplace=True)

file0 = f'{ym}_{brh}_수금수수료명세.xlsx'
with pd.ExcelWriter(dir0+file0,mode="w",engine='openpyxl') as writer:  
    df_sum.to_excel(writer, sheet_name='수금요약',index=False)
    df_all.to_excel(writer, sheet_name='수금명세',index=False)

print('end')


In [ ]:
# 피스토스 수금명세 추출
brh = '글래드피스토스'  #지점
ins = '장기'           #보종
fc_name = ["고경숙","홍미숙"]

ff = dir0+file0
df = pd.read_excel(ff, sheet_name='피스토스수금', header=3, usecols='a:l').iloc[:13]

col = ['보험사','증권번호','계약종류','지점','사원번호','사원명','FC코드','계약일자','계약자명']
df_all = pd.DataFrame()
for idx, row in df[['보험사','수금 값열','수금값 제목']].iterrows():
    cols = col + [ row['수금값 제목'] ]
    if row['보험사'] !='계':
        df_temp = (pd.read_excel(ff,sheet_name=row['보험사'], usecols=cols, dtype={'증권번호': 'object','계약일자': 'object'})
                  .rename(columns={row['수금값 제목']:'총괄수금수수료'}))
        if brh:
            df_temp = df_temp[df_temp['지점']==brh]
        if ins:    
            df_temp = df_temp[df_temp['계약종류']==ins]
        df_all = pd.concat([df_all, df_temp])
df_all['수금수수료']= df_all['총괄수금수수료'] * .95

# 피스토스 수금은 95%값
df_wrt1 = df_all.drop(columns='총괄수금수수료')
# 사원수금은 100%값을 추출함 : 정산 오류가 있었음
df_wrt2 = (df_all.query("계약일자 >= '20200801'  and  사원명 in (@fc_name) ")
            .drop(columns='수금수수료')
            .rename(columns={'총괄수금수수료':'수금수수료'}))

file0 = f'{ym}_수금수수료명세(피스토스).xlsx'
with pd.ExcelWriter(dir0+file0,mode="w",engine='openpyxl') as writer:  
    df_wrt1.to_excel(writer, sheet_name='피스토스',index=False)
    df_wrt2.to_excel(writer, sheet_name='사원(고경숙,홍미숙)',index=False)